# Notebook 04 — Validation & Interpretation (Stage 5 + SHAP)

## Verizon Cell Tower Lease Fair Market Value Estimation

This notebook runs all **5 defense layers** that validate the FMV predictions from Notebook 03,
and produces **SHAP-based interpretation artifacts** that decompose FMV into property-driven
vs. network-driven components.

### Defense Layer Overview

| Layer | Method | Purpose | Key Metric |
|-------|--------|---------|------------|
| **1** | Isolation Forest | Unsupervised anomaly detection cross-check | Cohen's Kappa |
| **2** | Local Outlier Factor | Density-based anomaly agreement | Three-way unanimous rate |
| **3** | External Benchmark Compliance | Check FMV against industry ranges | Compliance % |
| **4** | Land Value Correlation | State-level macro sanity check | Pearson r, Spearman \u03c1 |
| **5** | (Simulated) Expert Validation | Compare vs expert appraisals | MAPE, Kappa |

### SHAP Interpretation

- Beeswarm summary plot
- Feature importance bar chart
- X_MARKET vs Z_NETWORK decomposition (“X% property-driven, Y% network-driven”)
- Waterfall charts for 3 example sites
- Partial dependence plots for top 4 features

### Inputs
- `../data/processed/fmv_predictions.parquet` (output of Notebook 03)
- `../data/synthetic/synthetic_5k_sites.parquet` (fallback)

### Outputs
- Defense layer summary table
- Confidence heatmap identifying low-confidence regions
- Overall validation verdict

In [ ]:
# ============================================================
# 1  Imports & Configuration
# ============================================================
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import (
    cohen_kappa_score, confusion_matrix, classification_report,
    mean_absolute_error, r2_score, accuracy_score,
)
from sklearn.inspection import PartialDependenceDisplay

from xgboost import XGBRegressor

import shap

np.random.seed(42)
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 110
plt.rcParams['figure.facecolor'] = 'white'

print('All imports loaded successfully.')
print(f'SHAP version: {shap.__version__}')

---
## 1. Setup & Data Load

We load the FMV predictions from Notebook 03.  If that file is unavailable, we fall back
to the raw synthetic data and run a quick inline pipeline (clustering → pseudo-labeling →
XGBoost → QRF → classification) so the rest of the notebook can proceed.

In [ ]:
# ============================================================
# 1.1  Load data
# ============================================================
PRIMARY_PATH  = os.path.join('..', 'data', 'processed', 'fmv_predictions.parquet')
FALLBACK_PATH = os.path.join('..', 'data', 'synthetic', 'synthetic_5k_sites.parquet')

NEEDS_INLINE_PIPELINE = False

if os.path.exists(PRIMARY_PATH):
    df = pd.read_parquet(PRIMARY_PATH)
    print(f'\u2713 Loaded FMV predictions: {PRIMARY_PATH}')
    print(f'  Shape: {df.shape}')
elif os.path.exists(FALLBACK_PATH):
    df = pd.read_parquet(FALLBACK_PATH)
    print(f'\u2717 FMV predictions not found. Loaded fallback: {FALLBACK_PATH}')
    print(f'  Shape: {df.shape}')
    NEEDS_INLINE_PIPELINE = True
else:
    raise FileNotFoundError(
        'Neither fmv_predictions.parquet nor synthetic_5k_sites.parquet found.\n'
        'Please run Notebook 03 or Simulation 00 first.'
    )

print(f'\nKey columns present:')
for col in ['predicted_fmv', 'model_class', 'confidence', 'true_fmv', 'true_class']:
    status = '\u2713' if col in df.columns else '\u2717'
    print(f'  {status} {col}')

In [ ]:
# ============================================================
# 1.2  Inline pipeline fallback (runs only if fmv_predictions.parquet missing)
# ============================================================
if NEEDS_INLINE_PIPELINE:
    from scipy.stats import median_abs_deviation
    from sklearn.model_selection import GroupKFold
    from sklearn.linear_model import Ridge

    try:
        from quantile_forest import RandomForestQuantileRegressor
        HAS_QF = True
    except ImportError:
        HAS_QF = False

    print('Running inline FMV pipeline on fallback data...\n')

    # --- Quick clustering ---
    cluster_feats = [
        'census_population_density', 'census_median_income',
        'census_median_home_value', 'underlying_land_value_psf',
        'urban_rural_class', 'fcc_tower_count_5km',
        'tenant_count', 'current_monthly_rent',
    ]
    scaler_c = StandardScaler()
    X_c = scaler_c.fit_transform(df[cluster_feats])

    try:
        from sklearn.cluster import HDBSCAN as SkHDBSCAN
        labels = SkHDBSCAN(min_cluster_size=20, min_samples=5).fit_predict(X_c)
    except Exception:
        from sklearn.cluster import KMeans
        labels = KMeans(n_clusters=15, random_state=42, n_init=10).fit_predict(X_c)

    df['cluster_id'] = labels

    # --- Pseudo-labeling ---
    df['pseudo_label'] = 'FRINGE'
    for cid in sorted(df['cluster_id'].unique()):
        if cid == -1:
            df.loc[df['cluster_id'] == cid, 'pseudo_label'] = 'OUTLIER'
            continue
        mask = df['cluster_id'] == cid
        rents = df.loc[mask, 'current_monthly_rent'].values
        med = np.median(rents)
        mad = median_abs_deviation(rents, scale='normal')
        if mad < 1e-6:
            mad = max(np.std(rents), 1.0)
        z = np.abs(rents - med) / mad
        idx = df.index[mask]
        df.loc[idx[z <= 1.0], 'pseudo_label'] = 'CORE'
        df.loc[idx[z > 2.0], 'pseudo_label'] = 'OUTLIER'

    # --- Feature encoding ---
    X_MARKET = [
        'census_population_density', 'census_population_3km', 'census_median_income',
        'census_median_home_value', 'census_median_rent', 'census_vacancy_rate',
        'urban_rural_class', 'underlying_land_value_psf', 'hud_safmr',
        'property_tax_rate', 'topography_class', 'terrain_elevation_variance_m',
        'poi_density_1km', 'building_density_500m', 'distance_to_highway_km',
        'ground_elevation_ft', 'unemployment_rate_local',
        'fcc_tower_count_5km', 'fcc_tower_density_sqkm', 'competitor_tower_distance_km',
        'tower_per_capita', 'scarcity_index', 'landlord_concentration',
        'composite_hazard_score',
        'tower_height_ft', 'lease_area_sqft', 'parcel_acreage', 'construction_age',
        'site_access_ordinal', 'permit_approval_difficulty', 'wrluri_index',
        'colocation_required', 'height_restriction_ft',
        'has_building', 'is_ground_tower', 'has_parcel', 'has_zoning_data',
    ]
    Z_NETWORK = [
        'tenant_count', 'primary_technology', 'coverage_critical',
        'capacity_utilization_pct', 'high_traffic', 'backhaul_type',
        'has_fiber_connection', 'network_substitutability_score',
        'search_ring_alternative_count', 'inter_site_distance_km',
        'antenna_count', 'structure_capacity_remaining', 'site_priority_tier',
    ]
    CATEGORICALS = ['property_type', 'structure_type', 'zoning_district_type']

    enc_cols = []
    for col in CATEGORICALS:
        if col in df.columns:
            df[f'{col}_enc'] = LabelEncoder().fit_transform(df[col].astype(str))
            enc_cols.append(f'{col}_enc')

    ALL_FEATURES_ENC = [f for f in X_MARKET + Z_NETWORK + enc_cols if f in df.columns]

    # --- XGBoost + QRF + Meta-learner ---
    core_mask = df['pseudo_label'] == 'CORE'
    X_core = df.loc[core_mask, ALL_FEATURES_ENC].values.astype(np.float32)
    y_core = df.loc[core_mask, 'current_monthly_rent'].values.astype(np.float32)
    X_all_f = df[ALL_FEATURES_ENC].values.astype(np.float32)

    xgb_m = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6,
                         subsample=0.8, colsample_bytree=0.8, min_child_weight=5,
                         reg_alpha=0.1, reg_lambda=1.0, random_state=42,
                         n_jobs=-1, verbosity=0)
    xgb_m.fit(X_core, y_core, verbose=False)
    xgb_preds = xgb_m.predict(X_all_f)

    if HAS_QF:
        qrf_m = RandomForestQuantileRegressor(n_estimators=500, min_samples_leaf=5,
                                              random_state=42, n_jobs=-1)
        qrf_m.fit(X_core, y_core)
        qrf_preds = qrf_m.predict(X_all_f, quantiles=[0.10, 0.25, 0.50, 0.75, 0.90])
        for i, q in enumerate([0.10, 0.25, 0.50, 0.75, 0.90]):
            df[f'qrf_q{int(q*100):02d}'] = qrf_preds[:, i]
    else:
        rf_m = RandomForestRegressor(n_estimators=500, min_samples_leaf=5,
                                     random_state=42, n_jobs=-1)
        rf_m.fit(X_core, y_core)
        tree_p = np.array([t.predict(X_all_f) for t in rf_m.estimators_])
        for q in [0.10, 0.25, 0.50, 0.75, 0.90]:
            df[f'qrf_q{int(q*100):02d}'] = np.percentile(tree_p, q * 100, axis=0)

    # Meta-learner (simple average as quick fallback)
    df['xgb_pred'] = xgb_preds
    df['predicted_fmv'] = (xgb_preds + df['qrf_q50'].values) / 2
    df['predicted_fmv'] = df['predicted_fmv'].clip(lower=100, upper=15000)

    df['pi_width_iqr'] = df['qrf_q75'] - df['qrf_q25']
    df['pi_width_90'] = df['qrf_q90'] - df['qrf_q10']

    df['model_class'] = np.where(
        df['current_monthly_rent'] > df['qrf_q75'], 'OVERPAYING',
        np.where(df['current_monthly_rent'] < df['qrf_q25'], 'UNDERPAYING', 'AT_MARKET')
    )
    df['confidence'] = np.clip(
        1.0 - (df['pi_width_iqr'] / df['predicted_fmv'].clip(lower=1)), 0.0, 1.0
    )
    df['deviation_pct'] = (
        (df['current_monthly_rent'] - df['predicted_fmv']) / df['predicted_fmv'].clip(lower=1)
    )

    print(f'\u2713 Inline pipeline complete. Predictions for {len(df)} sites.')
    print(f'  Classification: {df["model_class"].value_counts().to_dict()}')
else:
    print('\u2713 FMV predictions loaded from Notebook 03 output.')

In [ ]:
# ============================================================
# 1.3  Reconstruct feature lists (needed for SHAP later)
# ============================================================
X_MARKET = [
    'census_population_density', 'census_population_3km', 'census_median_income',
    'census_median_home_value', 'census_median_rent', 'census_vacancy_rate',
    'urban_rural_class', 'underlying_land_value_psf', 'hud_safmr',
    'property_tax_rate', 'topography_class', 'terrain_elevation_variance_m',
    'poi_density_1km', 'building_density_500m', 'distance_to_highway_km',
    'ground_elevation_ft', 'unemployment_rate_local',
    'fcc_tower_count_5km', 'fcc_tower_density_sqkm', 'competitor_tower_distance_km',
    'tower_per_capita', 'scarcity_index', 'landlord_concentration',
    'composite_hazard_score',
    'tower_height_ft', 'lease_area_sqft', 'parcel_acreage', 'construction_age',
    'site_access_ordinal', 'permit_approval_difficulty', 'wrluri_index',
    'colocation_required', 'height_restriction_ft',
    'has_building', 'is_ground_tower', 'has_parcel', 'has_zoning_data',
]
Z_NETWORK = [
    'tenant_count', 'primary_technology', 'coverage_critical',
    'capacity_utilization_pct', 'high_traffic', 'backhaul_type',
    'has_fiber_connection', 'network_substitutability_score',
    'search_ring_alternative_count', 'inter_site_distance_km',
    'antenna_count', 'structure_capacity_remaining', 'site_priority_tier',
]
CATEGORICALS = ['property_type', 'structure_type', 'zoning_district_type']

# Ensure encoded categoricals exist
encoded_cat_cols = []
for col in CATEGORICALS:
    enc_col = f'{col}_enc'
    if enc_col in df.columns:
        encoded_cat_cols.append(enc_col)
    elif col in df.columns:
        df[enc_col] = LabelEncoder().fit_transform(df[col].astype(str))
        encoded_cat_cols.append(enc_col)

ALL_FEATURES_ENC = [f for f in X_MARKET + Z_NETWORK + encoded_cat_cols if f in df.columns]

print(f'Feature count: {len(ALL_FEATURES_ENC)} ({len([f for f in ALL_FEATURES_ENC if f in X_MARKET])} market, '
      f'{len([f for f in ALL_FEATURES_ENC if f in Z_NETWORK])} network, {len(encoded_cat_cols)} encoded cat)')

---
## Defense Layer 1: Isolation Forest

The **Isolation Forest (IF)** is an unsupervised anomaly detector that identifies sites whose
feature + rent combination is unusual.  We use it as a cross-check:

- Sites the IF flags as anomalous *should* tend to be classified as OVERPAYING or UNDERPAYING
  by our model (they are outliers in rent space).
- Sites the IF considers normal *should* tend to be AT_MARKET.

We measure agreement between IF anomaly labels and model classifications using **Cohen's Kappa**.
A kappa > 0.40 indicates moderate agreement, validating that the model’s classifications
align with an independent anomaly perspective.

In [ ]:
# ============================================================
# Defense Layer 1: Isolation Forest
# ============================================================

# Build feature matrix: ALL features + current rent (scaled)
if_features = ALL_FEATURES_ENC + ['current_monthly_rent']
X_if = df[if_features].values.astype(np.float32)

scaler_if = StandardScaler()
X_if_scaled = scaler_if.fit_transform(X_if)

# Train Isolation Forest
iso_forest = IsolationForest(
    n_estimators=500,
    contamination='auto',
    random_state=42,
    n_jobs=-1,
)
iso_forest.fit(X_if_scaled)

# Score anomalies: -1 = anomaly, 1 = normal
if_labels_raw = iso_forest.predict(X_if_scaled)
if_scores = iso_forest.decision_function(X_if_scaled)  # lower = more anomalous

# Determine directionality: anomalies that overpay vs underpay
# Use deviation from predicted FMV to assign direction
df['if_anomaly'] = (if_labels_raw == -1).astype(int)
df['if_score'] = if_scores

# Map IF results to classification-like labels for kappa comparison
# Anomalies with positive deviation -> OVERPAYING-like
# Anomalies with negative deviation -> UNDERPAYING-like
# Normal sites -> AT_MARKET-like
df['if_class'] = 'AT_MARKET'
anomaly_mask = df['if_anomaly'] == 1
df.loc[anomaly_mask & (df['deviation_pct'] > 0), 'if_class'] = 'OVERPAYING'
df.loc[anomaly_mask & (df['deviation_pct'] <= 0), 'if_class'] = 'UNDERPAYING'

# Cohen's Kappa
kappa_if = cohen_kappa_score(df['model_class'], df['if_class'])

# Agreement rate
agreement_if = (df['model_class'] == df['if_class']).mean()

print('Defense Layer 1: Isolation Forest')
print('=' * 50)
print(f'  Anomalies detected: {df["if_anomaly"].sum()} ({df["if_anomaly"].mean()*100:.1f}%)')
print(f'  IF classification distribution:')
for cls in ['OVERPAYING', 'AT_MARKET', 'UNDERPAYING']:
    n = (df['if_class'] == cls).sum()
    print(f'    {cls}: {n} ({n/len(df)*100:.1f}%)')
print(f'\n  Cohen\'s Kappa (model vs IF):  {kappa_if:.3f}')
print(f'  Agreement rate:               {agreement_if:.1%}')

# Interpretation
if kappa_if >= 0.60:
    interp = 'STRONG agreement -- IF strongly validates model classifications'
elif kappa_if >= 0.40:
    interp = 'MODERATE agreement -- IF provides reasonable validation'
elif kappa_if >= 0.20:
    interp = 'FAIR agreement -- partial alignment between model and IF'
else:
    interp = 'WEAK agreement -- model and IF disagree substantially'
print(f'  Interpretation: {interp}')

In [ ]:
# ============================================================
# Defense Layer 1: Confusion Matrix (Model vs IF)
# ============================================================
class_labels = ['UNDERPAYING', 'AT_MARKET', 'OVERPAYING']

cm_if = confusion_matrix(df['model_class'], df['if_class'], labels=class_labels)

fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(
    cm_if, annot=True, fmt='d', cmap='YlOrRd',
    xticklabels=class_labels, yticklabels=class_labels,
    ax=ax, cbar_kws={'label': 'Count'},
    linewidths=0.5, linecolor='white',
)
ax.set_xlabel('Isolation Forest Label', fontsize=11)
ax.set_ylabel('Model Classification', fontsize=11)
ax.set_title(f'Defense Layer 1: Model vs Isolation Forest\n(Kappa = {kappa_if:.3f})',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

---
## Defense Layer 2: Local Outlier Factor

**LOF** is a density-based anomaly detector that identifies sites living in low-density
regions of feature space.  Unlike IF (which uses random splits), LOF explicitly measures
how isolated a point is relative to its k-nearest neighbors.

We compute:
1. Kappa with model classification
2. **Three-way agreement** rate: model + IF + LOF all agree
3. **Confidence tiers**: HIGH (3/3 agree), MEDIUM (2/3), LOW (0–1/3)

In [ ]:
# ============================================================
# Defense Layer 2: Local Outlier Factor
# ============================================================

# LOF uses the same scaled feature matrix as IF
lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination='auto',
    novelty=False,
    n_jobs=-1,
)
lof_labels_raw = lof.fit_predict(X_if_scaled)  # -1 = outlier, 1 = inlier
lof_scores = lof.negative_outlier_factor_  # more negative = more anomalous

df['lof_anomaly'] = (lof_labels_raw == -1).astype(int)
df['lof_score'] = lof_scores

# Map LOF anomalies to directional labels (same logic as IF)
df['lof_class'] = 'AT_MARKET'
lof_anomaly_mask = df['lof_anomaly'] == 1
df.loc[lof_anomaly_mask & (df['deviation_pct'] > 0), 'lof_class'] = 'OVERPAYING'
df.loc[lof_anomaly_mask & (df['deviation_pct'] <= 0), 'lof_class'] = 'UNDERPAYING'

# Kappa with model
kappa_lof = cohen_kappa_score(df['model_class'], df['lof_class'])

# Three-way agreement: model + IF + LOF
three_way_agree = (
    (df['model_class'] == df['if_class']) & (df['model_class'] == df['lof_class'])
).mean()

# Count agreements per site (how many of 3 methods agree on the model's label)
df['n_agree'] = (
    (df['model_class'] == df['if_class']).astype(int) +
    (df['model_class'] == df['lof_class']).astype(int) +
    1  # model always agrees with itself
)

# Confidence tiers
df['defense_tier'] = np.where(
    df['n_agree'] == 3, 'HIGH',
    np.where(df['n_agree'] == 2, 'MEDIUM', 'LOW')
)

print('Defense Layer 2: Local Outlier Factor')
print('=' * 50)
print(f'  LOF anomalies: {df["lof_anomaly"].sum()} ({df["lof_anomaly"].mean()*100:.1f}%)')
print(f'  Cohen\'s Kappa (model vs LOF):  {kappa_lof:.3f}')
print(f'\n  Three-way agreement (model + IF + LOF): {three_way_agree:.1%}')
print(f'\n  Confidence tier distribution:')
for tier in ['HIGH', 'MEDIUM', 'LOW']:
    n = (df['defense_tier'] == tier).sum()
    print(f'    {tier:6s}: {n:5d} ({n/len(df)*100:.1f}%)')

---
## Defense Layer 3: External Benchmark Compliance

We define **benchmark FMV ranges** by market segment (urban/suburban/rural) and property type,
based on industry knowledge of typical cell tower lease rates.  We then check what fraction
of our predicted FMVs fall within a **30% tolerance** band of these benchmarks.

A compliance rate above 70% indicates the model produces economically reasonable estimates.

In [ ]:
# ============================================================
# Defense Layer 3: External Benchmark Compliance
# ============================================================

# Define benchmark ranges: (low, high) monthly rent in USD
# Source: industry reports, CBRE/JLL tower lease surveys, expert judgment
benchmark_ranges = {
    # (urban_rural_class, property_type) -> (low, high)
    (1, 'commercial'):    (1500, 3500),
    (1, 'rooftop'):       (1800, 4000),
    (1, 'industrial'):    (1200, 3000),
    (1, 'residential'):   (1000, 2800),
    (1, 'government'):    (1200, 3200),
    (1, 'vacant_land'):   (800, 2500),
    (1, 'agricultural'):  (600, 2000),
    (2, 'commercial'):    (1000, 2500),
    (2, 'rooftop'):       (1200, 3000),
    (2, 'industrial'):    (800, 2200),
    (2, 'residential'):   (700, 2000),
    (2, 'government'):    (800, 2200),
    (2, 'vacant_land'):   (500, 1800),
    (2, 'agricultural'):  (400, 1500),
    (3, 'commercial'):    (600, 1800),
    (3, 'rooftop'):       (700, 2000),
    (3, 'industrial'):    (500, 1500),
    (3, 'residential'):   (400, 1400),
    (3, 'government'):    (500, 1600),
    (3, 'vacant_land'):   (300, 1200),
    (3, 'agricultural'):  (200, 1000),
}

TOLERANCE = 0.30  # 30% tolerance band

# Check compliance for each site
compliant_flags = []
benchmark_status = []  # 'within', 'above', 'below'

for _, row in df.iterrows():
    urc = row.get('urban_rural_class', 2)
    pt = row.get('property_type', 'commercial')
    fmv = row['predicted_fmv']

    key = (int(urc), str(pt))
    if key in benchmark_ranges:
        low, high = benchmark_ranges[key]
    else:
        # Default benchmark if segment not defined
        low, high = 500, 2500

    # Apply tolerance
    low_tol = low * (1 - TOLERANCE)
    high_tol = high * (1 + TOLERANCE)

    if low_tol <= fmv <= high_tol:
        compliant_flags.append(True)
        benchmark_status.append('within')
    elif fmv > high_tol:
        compliant_flags.append(False)
        benchmark_status.append('above')
    else:
        compliant_flags.append(False)
        benchmark_status.append('below')

df['benchmark_compliant'] = compliant_flags
df['benchmark_status'] = benchmark_status

compliance_rate = df['benchmark_compliant'].mean()
above_pct = (df['benchmark_status'] == 'above').mean()
below_pct = (df['benchmark_status'] == 'below').mean()

print('Defense Layer 3: External Benchmark Compliance')
print('=' * 50)
print(f'  Tolerance band: \u00b1{TOLERANCE*100:.0f}%')
print(f'  Compliance rate: {compliance_rate:.1%}')
print(f'  Above benchmark: {above_pct:.1%}')
print(f'  Below benchmark: {below_pct:.1%}')
print(f'\n  Target: \u226570% compliance')
compliance_pass = compliance_rate >= 0.70
print(f'  Result: {"\u2713 PASS" if compliance_pass else "\u2717 FAIL"}')

In [ ]:
# ============================================================
# Defense Layer 3: Compliance by Market Segment (bar chart)
# ============================================================

# Build segment labels
urc_labels = {1: 'Urban', 2: 'Suburban', 3: 'Rural'}
if 'urban_rural_class' in df.columns and 'property_type' in df.columns:
    df['market_segment'] = (
        df['urban_rural_class'].map(urc_labels).fillna('Unknown') + ' / ' +
        df['property_type'].astype(str).str.replace('_', ' ').str.title()
    )
else:
    df['market_segment'] = 'Unknown'

# Compliance by segment
seg_compliance = (
    df.groupby('market_segment')['benchmark_compliant']
    .agg(['mean', 'count'])
    .rename(columns={'mean': 'compliance', 'count': 'n_sites'})
    .sort_values('compliance', ascending=True)
)
# Filter to segments with at least 20 sites
seg_compliance = seg_compliance[seg_compliance['n_sites'] >= 20]

fig, ax = plt.subplots(figsize=(10, max(6, len(seg_compliance) * 0.4)))
colors = ['#4CAF50' if c >= 0.70 else '#FF9800' if c >= 0.50 else '#F44336'
          for c in seg_compliance['compliance']]

bars = ax.barh(
    range(len(seg_compliance)),
    seg_compliance['compliance'] * 100,
    color=colors, edgecolor='white', linewidth=0.5,
)
ax.set_yticks(range(len(seg_compliance)))
ax.set_yticklabels(seg_compliance.index, fontsize=9)
ax.axvline(70, color='red', ls='--', lw=1.5, alpha=0.7, label='70% target')
ax.set_xlabel('Benchmark Compliance (%)', fontsize=11)
ax.set_title('Defense Layer 3: Benchmark Compliance by Market Segment',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=9)

# Annotate counts
for i, (_, row) in enumerate(seg_compliance.iterrows()):
    ax.text(row['compliance'] * 100 + 1, i, f'n={row["n_sites"]:.0f}',
            va='center', fontsize=8, color='gray')

plt.tight_layout()
plt.show()

---
## Defense Layer 4: Land Value Correlation

Cell tower lease FMVs should correlate with **underlying land values** at the state level.
States with higher land values (e.g., CA, NY, NJ) should have higher median predicted FMVs.

We compute **Pearson r** and **Spearman \u03c1** between state-level median land value and
state-level median predicted FMV.  A Pearson r > 0.40 indicates the model captures
macro-geographic value gradients correctly.

In [ ]:
# ============================================================
# Defense Layer 4: Land Value Correlation
# ============================================================

# State-level aggregation
state_agg = df.groupby('state').agg(
    median_land_value=('underlying_land_value_psf', 'median'),
    median_fmv=('predicted_fmv', 'median'),
    mean_fmv=('predicted_fmv', 'mean'),
    n_sites=('site_id', 'count'),
).reset_index()

# Filter to states with enough sites for stable estimates
MIN_SITES = 5
state_agg_filtered = state_agg[state_agg['n_sites'] >= MIN_SITES].copy()

# Pearson and Spearman correlations
if len(state_agg_filtered) >= 5:
    r_pearson, p_pearson = pearsonr(
        state_agg_filtered['median_land_value'],
        state_agg_filtered['median_fmv']
    )
    r_spearman, p_spearman = spearmanr(
        state_agg_filtered['median_land_value'],
        state_agg_filtered['median_fmv']
    )
else:
    r_pearson = r_spearman = p_pearson = p_spearman = np.nan

print('Defense Layer 4: Land Value Correlation (State Level)')
print('=' * 50)
print(f'  States with \u2265{MIN_SITES} sites: {len(state_agg_filtered)}')
print(f'  Pearson r:  {r_pearson:.3f} (p={p_pearson:.4f})')
print(f'  Spearman \u03c1: {r_spearman:.3f} (p={p_spearman:.4f})')
print(f'\n  Target: Pearson r \u2265 0.40')
print(f'  Result: {"\u2713 PASS" if r_pearson >= 0.40 else "\u2717 FAIL"}')

In [ ]:
# ============================================================
# Defense Layer 4: Scatter Plot
# ============================================================
fig, ax = plt.subplots(figsize=(9, 7))

scatter = ax.scatter(
    state_agg_filtered['median_land_value'],
    state_agg_filtered['median_fmv'],
    s=state_agg_filtered['n_sites'] * 3,  # size by count
    c='#2196F3', alpha=0.7, edgecolors='white', linewidth=0.5,
)

# Label top states
for _, row in state_agg_filtered.nlargest(8, 'median_fmv').iterrows():
    ax.annotate(
        row['state'], (row['median_land_value'], row['median_fmv']),
        fontsize=8, alpha=0.8, ha='left',
        xytext=(5, 5), textcoords='offset points',
    )

# Regression line
if len(state_agg_filtered) >= 3:
    z = np.polyfit(state_agg_filtered['median_land_value'],
                   state_agg_filtered['median_fmv'], 1)
    p_line = np.poly1d(z)
    x_range = np.linspace(state_agg_filtered['median_land_value'].min(),
                          state_agg_filtered['median_land_value'].max(), 100)
    ax.plot(x_range, p_line(x_range), 'r--', lw=1.5, alpha=0.7,
            label=f'OLS fit (r={r_pearson:.2f})')

ax.set_xlabel('State Median Land Value ($/sqft)', fontsize=11)
ax.set_ylabel('State Median Predicted FMV ($/mo)', fontsize=11)
ax.set_title(f'Defense Layer 4: Land Value vs Predicted FMV\n'
             f'(Pearson r={r_pearson:.3f}, Spearman \u03c1={r_spearman:.3f})',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=9)

plt.tight_layout()
plt.show()

---
## Defense Layer 5: (Simulated) Expert Validation

In production, we would compare model FMVs against independent expert appraisals.
Since we’re working with synthetic data, we **simulate** expert estimates by adding
12% Gaussian noise to the true FMV (reflecting real-world appraiser disagreement).

We measure:
- MAPE between model predictions and expert estimates
- Cohen’s Kappa on classifications
- A 2\u00d72 comparison table for sample sites

In [ ]:
# ============================================================
# Defense Layer 5: (Simulated) Expert Validation
# ============================================================

if 'true_fmv' in df.columns:
    # Simulate expert estimates: true_fmv + 12% noise
    expert_noise = np.random.normal(0, 0.12, size=len(df))
    df['expert_fmv'] = df['true_fmv'] * (1 + expert_noise)
    df['expert_fmv'] = df['expert_fmv'].clip(lower=100)

    # Expert classification: compare current rent to expert FMV
    expert_tolerance = 0.20  # 20% band for expert classification
    df['expert_class'] = np.where(
        df['current_monthly_rent'] > df['expert_fmv'] * (1 + expert_tolerance), 'OVERPAYING',
        np.where(
            df['current_monthly_rent'] < df['expert_fmv'] * (1 - expert_tolerance), 'UNDERPAYING',
            'AT_MARKET'
        )
    )

    # MAPE between model and expert
    expert_mape = np.mean(
        np.abs(df['predicted_fmv'] - df['expert_fmv']) /
        np.maximum(df['expert_fmv'], 1)
    ) * 100

    # Kappa on classifications
    kappa_expert = cohen_kappa_score(df['model_class'], df['expert_class'])
    agreement_expert = (df['model_class'] == df['expert_class']).mean()

    print('Defense Layer 5: Simulated Expert Validation')
    print('=' * 50)
    print(f'  Expert noise level: \u00b112%')
    print(f'  MAPE (model vs expert): {expert_mape:.1f}%')
    print(f'  Cohen\'s Kappa (class): {kappa_expert:.3f}')
    print(f'  Agreement rate:        {agreement_expert:.1%}')

    # Sample comparison table
    print(f'\n  Sample site comparison (5 random sites):')
    sample_cols = ['site_id', 'current_monthly_rent', 'predicted_fmv',
                   'expert_fmv', 'model_class', 'expert_class']
    sample_idx = np.random.choice(len(df), size=5, replace=False)
    print(df.iloc[sample_idx][sample_cols].to_string(index=False))
else:
    expert_mape = np.nan
    kappa_expert = np.nan
    print('\u26a0 No true_fmv column -- cannot simulate expert validation.')
    print('  In production, this layer uses actual expert appraisals.')

In [ ]:
# ============================================================
# Defense Layer 5: 2x2 Comparison for Sample Sites
# ============================================================
if 'expert_fmv' in df.columns:
    # Pick 2 overpaying, 2 at-market, 2 underpaying
    example_sites = pd.concat([
        df[df['model_class'] == 'OVERPAYING'].sample(2, random_state=42),
        df[df['model_class'] == 'AT_MARKET'].sample(2, random_state=42),
        df[df['model_class'] == 'UNDERPAYING'].sample(2, random_state=42),
    ])

    comparison_cols = [
        'site_id', 'state', 'property_type', 'current_monthly_rent',
        'predicted_fmv', 'expert_fmv', 'model_class', 'expert_class', 'confidence',
    ]
    comparison_cols = [c for c in comparison_cols if c in df.columns]

    print('Model vs Expert: Detailed Comparison (6 sites)\n')
    display_df = example_sites[comparison_cols].copy()
    for col in ['current_monthly_rent', 'predicted_fmv', 'expert_fmv']:
        if col in display_df.columns:
            display_df[col] = display_df[col].apply(lambda x: f'${x:,.0f}')
    if 'confidence' in display_df.columns:
        display_df['confidence'] = display_df['confidence'].apply(lambda x: f'{x:.2f}')
    print(display_df.to_string(index=False))

---
## SHAP Interpretation

We use **SHAP (SHapley Additive exPlanations)** to understand how each feature contributes
to the predicted FMV.  SHAP values have a crucial property: they decompose each prediction
into additive feature contributions, allowing us to:

1. Rank features by their average impact on FMV
2. Quantify the **property-driven vs. network-driven** split
3. Show **individual site explanations** via waterfall charts
4. Identify non-linear feature effects via partial dependence

We retrain XGBoost on all CORE sites specifically for SHAP analysis.

In [ ]:
# ============================================================
# 7.1  Retrain XGBoost for SHAP
# ============================================================

# Use pseudo_label if available, otherwise use all data
if 'pseudo_label' in df.columns:
    core_mask = df['pseudo_label'] == 'CORE'
else:
    core_mask = pd.Series(True, index=df.index)

X_core_shap = df.loc[core_mask, ALL_FEATURES_ENC].values.astype(np.float32)
y_core_shap = df.loc[core_mask, 'current_monthly_rent'].values.astype(np.float32)

# Feature names DataFrame for SHAP display
X_core_df = pd.DataFrame(X_core_shap, columns=ALL_FEATURES_ENC)

# Train XGBoost
xgb_shap = XGBRegressor(
    n_estimators=500, learning_rate=0.05, max_depth=6,
    subsample=0.8, colsample_bytree=0.8, min_child_weight=5,
    reg_alpha=0.1, reg_lambda=1.0, random_state=42,
    n_jobs=-1, verbosity=0,
)
xgb_shap.fit(X_core_shap, y_core_shap, verbose=False)

print(f'\u2713 XGBoost retrained for SHAP on {len(X_core_shap)} CORE sites')
print(f'  Features: {len(ALL_FEATURES_ENC)}')

In [ ]:
# ============================================================
# 7.2  Compute SHAP values
# ============================================================

explainer = shap.TreeExplainer(xgb_shap)

# Compute SHAP values for all sites (not just core)
X_all_df = pd.DataFrame(
    df[ALL_FEATURES_ENC].values.astype(np.float32),
    columns=ALL_FEATURES_ENC
)
shap_values = explainer(X_all_df)

print(f'\u2713 SHAP values computed for {len(X_all_df)} sites')
print(f'  SHAP values shape: {shap_values.values.shape}')
print(f'  Base value (E[f(x)]): ${explainer.expected_value:.0f}/mo')

In [ ]:
# ============================================================
# 7.3  SHAP Summary Plot (Beeswarm)
# ============================================================
fig, ax = plt.subplots(figsize=(10, 10))
shap.summary_plot(
    shap_values.values,
    X_all_df,
    max_display=20,
    show=False,
)
plt.title('SHAP Beeswarm: Feature Impact on Predicted FMV', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 7.4  SHAP Feature Importance Bar Chart
# ============================================================
fig, ax = plt.subplots(figsize=(10, 8))
shap.plots.bar(
    shap_values,
    max_display=20,
    show=False,
)
plt.title('SHAP Feature Importance (mean |SHAP|)', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 7.5  SHAP Decomposition: X_MARKET vs Z_NETWORK
# ============================================================

# Identify column indices belonging to each domain
x_market_set = set(X_MARKET)
z_network_set = set(Z_NETWORK)

x_market_idx = [i for i, f in enumerate(ALL_FEATURES_ENC) if f in x_market_set]
z_network_idx = [i for i, f in enumerate(ALL_FEATURES_ENC) if f in z_network_set]
other_idx = [i for i in range(len(ALL_FEATURES_ENC))
             if i not in x_market_idx and i not in z_network_idx]

# Sum absolute SHAP values per domain, per site
sv = shap_values.values  # (n_samples, n_features)

market_shap = np.abs(sv[:, x_market_idx]).sum(axis=1)   # per-site market contribution
network_shap = np.abs(sv[:, z_network_idx]).sum(axis=1)  # per-site network contribution
other_shap = np.abs(sv[:, other_idx]).sum(axis=1) if other_idx else np.zeros(len(sv))
total_shap = market_shap + network_shap + other_shap

# Average across all sites
market_pct = (market_shap / total_shap).mean() * 100
network_pct = (network_shap / total_shap).mean() * 100
other_pct = (other_shap / total_shap).mean() * 100 if other_idx else 0

print('SHAP Decomposition: Property vs Network Drivers')
print('=' * 55)
print(f'  X_MARKET  (property/location):  {market_pct:.1f}%')
print(f'  Z_NETWORK (carrier/network):    {network_pct:.1f}%')
if other_pct > 0.1:
    print(f'  Other (encoded categoricals):   {other_pct:.1f}%')

print(f'\n  \u2192 On average, {market_pct:.0f}% of FMV is property-driven, '
      f'{network_pct:.0f}% is network-driven.')

# Pie chart
fig, ax = plt.subplots(figsize=(7, 7))
sizes = [market_pct, network_pct]
labels_pie = [f'X_MARKET\n(Property/Location)\n{market_pct:.1f}%',
              f'Z_NETWORK\n(Carrier/Network)\n{network_pct:.1f}%']
colors_pie = ['#2196F3', '#FF9800']
if other_pct > 0.5:
    sizes.append(other_pct)
    labels_pie.append(f'Other\n{other_pct:.1f}%')
    colors_pie.append('#9C27B0')

wedges, texts = ax.pie(
    sizes, labels=labels_pie, colors=colors_pie,
    startangle=90, textprops={'fontsize': 11},
    wedgeprops={'edgecolor': 'white', 'linewidth': 2},
)
ax.set_title('SHAP Decomposition: Property-Driven vs Network-Driven FMV',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 7.6  Waterfall Charts for 3 Example Sites
# ============================================================

# Select 1 overpaying, 1 at-market, 1 underpaying
example_indices = {}
for cls in ['OVERPAYING', 'AT_MARKET', 'UNDERPAYING']:
    cls_mask = df['model_class'] == cls
    if cls_mask.any():
        # Pick site closest to median confidence for representativeness
        cls_df = df[cls_mask]
        median_conf = cls_df['confidence'].median()
        idx = (cls_df['confidence'] - median_conf).abs().idxmin()
        example_indices[cls] = df.index.get_loc(idx)

for cls, idx in example_indices.items():
    site_row = df.iloc[idx]
    print(f'\n--- {cls} Example ---')
    print(f'Site: {site_row.get("site_id", idx)}, State: {site_row.get("state", "?")}, '
          f'Property: {site_row.get("property_type", "?")}')
    print(f'Current Rent: ${site_row["current_monthly_rent"]:,.0f}, '
          f'Predicted FMV: ${site_row["predicted_fmv"]:,.0f}, '
          f'Confidence: {site_row["confidence"]:.2f}')

    fig, ax = plt.subplots(figsize=(10, 6))
    shap.plots.waterfall(shap_values[idx], max_display=12, show=False)
    plt.title(f'SHAP Waterfall: {cls} Site ({site_row.get("site_id", idx)})',
              fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()

In [ ]:
# ============================================================
# 7.7  Partial Dependence Plots for Top 4 Features (2x2 grid)
# ============================================================

# Identify top 4 features by mean |SHAP|
mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
top4_indices = np.argsort(mean_abs_shap)[-4:][::-1]
top4_features = [ALL_FEATURES_ENC[i] for i in top4_indices]

print(f'Top 4 features for PDP: {top4_features}\n')

# SHAP dependence plots (feature value vs SHAP value) with binned trend
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax_i, (feat, ax) in enumerate(zip(top4_features, axes.ravel())):
    feat_idx = ALL_FEATURES_ENC.index(feat)

    # Scatter: feature value vs SHAP value
    ax.scatter(
        X_all_df[feat].values,
        shap_values.values[:, feat_idx],
        alpha=0.15, s=4, c='#2196F3',
    )

    # Add LOESS-like trend (binned mean)
    feat_vals = X_all_df[feat].values
    shap_vals = shap_values.values[:, feat_idx]
    n_bins = 30
    bins = np.linspace(np.percentile(feat_vals, 2), np.percentile(feat_vals, 98), n_bins)
    bin_indices = np.digitize(feat_vals, bins)
    bin_means_x = []
    bin_means_y = []
    for b in range(1, len(bins)):
        mask = bin_indices == b
        if mask.sum() > 5:
            bin_means_x.append(np.mean(feat_vals[mask]))
            bin_means_y.append(np.mean(shap_vals[mask]))
    ax.plot(bin_means_x, bin_means_y, 'r-', lw=2, alpha=0.8, label='Binned mean')

    ax.axhline(0, color='gray', ls=':', lw=0.8)
    ax.set_xlabel(feat, fontsize=10)
    ax.set_ylabel('SHAP value ($/mo)', fontsize=10)
    ax.set_title(f'PDP: {feat}', fontsize=11, fontweight='bold')
    ax.legend(fontsize=8)

plt.suptitle('Partial Dependence Plots (Top 4 Features by SHAP Importance)',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---
## 8. Confidence Heatmap

We plot a **geographic heatmap** of model confidence scores to identify regions where
the model is less certain.  Low-confidence regions are candidates for:
- Additional data collection
- Manual expert appraisals
- Targeted validation efforts

In [ ]:
# ============================================================
# 8.  Confidence Heatmap (Geographic)
# ============================================================

has_coords = 'longitude' in df.columns and 'latitude' in df.columns

if has_coords:
    fig, axes = plt.subplots(1, 2, figsize=(18, 7))

    # --- Confidence heatmap ---
    sc1 = axes[0].scatter(
        df['longitude'], df['latitude'],
        c=df['confidence'], cmap='RdYlGn', s=6, alpha=0.7,
        vmin=0, vmax=1,
    )
    plt.colorbar(sc1, ax=axes[0], label='Confidence Score', shrink=0.8)
    axes[0].set_xlabel('Longitude', fontsize=10)
    axes[0].set_ylabel('Latitude', fontsize=10)
    axes[0].set_title('Model Confidence by Location', fontsize=12, fontweight='bold')

    # --- Classification map ---
    cls_colors_map = {'OVERPAYING': '#F44336', 'AT_MARKET': '#4CAF50', 'UNDERPAYING': '#2196F3'}
    for cls in ['AT_MARKET', 'UNDERPAYING', 'OVERPAYING']:
        mask = df['model_class'] == cls
        axes[1].scatter(
            df.loc[mask, 'longitude'], df.loc[mask, 'latitude'],
            c=cls_colors_map[cls], s=6, alpha=0.5, label=cls,
        )
    axes[1].set_xlabel('Longitude', fontsize=10)
    axes[1].set_ylabel('Latitude', fontsize=10)
    axes[1].set_title('Classification by Location', fontsize=12, fontweight='bold')
    axes[1].legend(fontsize=9, loc='lower left', markerscale=3)

    plt.suptitle('Geographic Distribution of Confidence & Classification',
                 fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

    # Identify lowest-confidence regions
    low_conf_threshold = df['confidence'].quantile(0.10)
    low_conf = df[df['confidence'] <= low_conf_threshold]

    if 'state' in df.columns:
        low_conf_states = (
            low_conf.groupby('state').size()
            .sort_values(ascending=False).head(10)
        )
        print(f'\nLowest-confidence regions (bottom 10% of confidence, threshold={low_conf_threshold:.2f}):')
        print(f'  Total low-confidence sites: {len(low_conf)}')
        print(f'\n  Top states with low-confidence sites:')
        for state, count in low_conf_states.items():
            print(f'    {state}: {count} sites')
        print(f'\n  \u2192 These regions are candidates for manual appraisals.')
else:
    print('\u26a0 No longitude/latitude columns -- showing confidence distribution instead.')

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.hist(df['confidence'], bins=50, color='#2196F3', edgecolor='white', alpha=0.85)
    ax.axvline(df['confidence'].median(), color='red', ls='--', lw=1.5,
               label=f'Median: {df["confidence"].median():.2f}')
    ax.set_xlabel('Confidence Score', fontsize=11)
    ax.set_ylabel('Count', fontsize=11)
    ax.set_title('Distribution of Model Confidence Scores', fontsize=13, fontweight='bold')
    ax.legend(fontsize=9)
    plt.tight_layout()
    plt.show()

---
## 9. Summary

We compile all defense layer results into a summary table and issue an overall
validation verdict.

In [ ]:
# ============================================================
# 9.  Defense Layer Summary Table
# ============================================================

summary_data = [
    {
        'Layer': '1. Isolation Forest',
        'Metric': "Cohen's Kappa",
        'Value': f'{kappa_if:.3f}',
        'Target': '\u2265 0.40',
        'Result': '\u2713 PASS' if kappa_if >= 0.40 else '\u2717 FAIL',
    },
    {
        'Layer': '2. Local Outlier Factor',
        'Metric': '3-Way Agreement',
        'Value': f'{three_way_agree:.1%}',
        'Target': '\u2265 50%',
        'Result': '\u2713 PASS' if three_way_agree >= 0.50 else '\u2717 FAIL',
    },
    {
        'Layer': '3. Benchmark Compliance',
        'Metric': 'Compliance Rate',
        'Value': f'{compliance_rate:.1%}',
        'Target': '\u2265 70%',
        'Result': '\u2713 PASS' if compliance_rate >= 0.70 else '\u2717 FAIL',
    },
    {
        'Layer': '4. Land Value Correlation',
        'Metric': 'Pearson r',
        'Value': f'{r_pearson:.3f}',
        'Target': '\u2265 0.40',
        'Result': '\u2713 PASS' if r_pearson >= 0.40 else '\u2717 FAIL',
    },
    {
        'Layer': '5. Expert Validation',
        'Metric': 'MAPE (model vs expert)',
        'Value': f'{expert_mape:.1f}%' if not np.isnan(expert_mape) else 'N/A',
        'Target': '\u2264 20%',
        'Result': ('\u2713 PASS' if expert_mape <= 20 else '\u2717 FAIL')
                  if not np.isnan(expert_mape) else 'N/A',
    },
]

summary_df = pd.DataFrame(summary_data)

print('\n' + '=' * 75)
print('DEFENSE LAYER SUMMARY')
print('=' * 75)
print(summary_df.to_string(index=False))

# Count passes
n_pass = sum(1 for r in summary_data if '\u2713' in r['Result'])
n_total = sum(1 for r in summary_data if r['Result'] != 'N/A')

print(f'\n{"="*75}')
print(f'Layers passed: {n_pass}/{n_total}')

In [ ]:
# ============================================================
# 9.2  Overall Validation Verdict
# ============================================================

print('\n' + '=' * 75)
print('OVERALL VALIDATION VERDICT')
print('=' * 75)

if n_pass >= 4:
    verdict = 'STRONG VALIDATION'
    verdict_detail = (
        'The FMV model passes the majority of defense layers.  Predictions are\n'
        'consistent with unsupervised anomaly detection, industry benchmarks,\n'
        'macro-geographic land value gradients, and (simulated) expert opinion.\n'
        'The model is suitable for production use with standard monitoring.'
    )
elif n_pass >= 3:
    verdict = 'MODERATE VALIDATION'
    verdict_detail = (
        'The FMV model passes most defense layers but has some gaps.\n'
        'Review failing layers for potential model improvements.\n'
        'Consider additional expert validation before production deployment.'
    )
else:
    verdict = 'WEAK VALIDATION'
    verdict_detail = (
        'The FMV model fails multiple defense layers.\n'
        'Significant model refinement is recommended before deployment.\n'
        'Investigate root causes of defense layer failures.'
    )

print(f'\n  Verdict: {verdict} ({n_pass}/{n_total} layers passed)')
print(f'\n  {verdict_detail}')

# SHAP decomposition summary
print(f'\n  SHAP Decomposition:')
print(f'    Property-driven (X_MARKET):  {market_pct:.1f}%')
print(f'    Network-driven (Z_NETWORK):  {network_pct:.1f}%')

# Confidence summary
print(f'\n  Confidence Distribution:')
for tier in ['HIGH', 'MEDIUM', 'LOW']:
    n = (df['defense_tier'] == tier).sum()
    print(f'    {tier:6s}: {n:5d} ({n/len(df)*100:.1f}%)')

print(f'\n  Mean model confidence: {df["confidence"].mean():.3f}')
print(f'  Sites needing review (LOW tier): {(df["defense_tier"] == "LOW").sum()}')

print(f'\n{"="*75}')
print(f'\u2713 Notebook 04 complete -- Validation & Interpretation pipeline finished.')
print(f'\u2713 Ready for Notebook 05 (Forecasting) or final reporting.')